In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt

root_path = '/var/datasets/single_papers/2023_feargal/GSE199750'


In [ ]:
metadata_df = read_prefixes_from_metadata_txt(f'{root_path}/GSE199750_series_matrix.txt')
print(metadata_df.shape)
metadata_df.head()

In [ ]:
input_dir = f'{root_path}/GSE199750_RAW'
count_matrix = pd.DataFrame()


for file in os.listdir(input_dir):
    if file.endswith(".featureCounts.txt"):
        file_path = os.path.join(input_dir, file)
        data = pd.read_csv(file_path, comment='#', sep='\t')
        gene_counts = data[['Geneid', data.columns[-1]]]  # Last column is the count column
        sample_name = os.path.splitext(file)[0]
        gene_counts.columns = ['Geneid', sample_name]
        if count_matrix.empty:
            count_matrix = gene_counts
        else:
            count_matrix = pd.merge(count_matrix, gene_counts, on='Geneid', how='outer')
count_matrix.set_index('Geneid', inplace=True)

        

In [ ]:
count_matrix_T = count_matrix.T
print(count_matrix_T.shape)
count_matrix_T.head()